<a href="https://colab.research.google.com/github/ascentadmin/pythonProject/blob/master/UpScale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Install modules for downloading our SDXL models
import requests
from tqdm.notebook import tqdm
import os

import tensorflow as tf

# Check TensorFlow GPU availability
print("TensorFlow version:", tf.__version__)
print("Is GPU available? ", tf.test.is_gpu_available())

# List available GPUs
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

def mount_gdrive():
    try:
        from google.colab import drive
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive successfully mounted.")
    except Exception as e:
        print(f"An error occurred while mounting Google Drive: {e}")

def download_file_from_url(url, folder_path, user_filename):
    try:
        # Full path to save the file
        save_path = os.path.join(folder_path, user_filename)

        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code

        # Get the total file size
        file_size = int(response.headers.get('content-length', 0))

        # Create a progress bar
        progress_bar = tqdm(total=file_size, unit='iB', unit_scale=True)

        # Open the file to write the content
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                progress_bar.update(len(chunk))
                file.write(chunk)

        progress_bar.close()

        # Check if the file size is correct
        if file_size != 0 and file_size != progress_bar.n:
            print("ERROR: Something went wrong with the download. Please try again.")
        else:
            print(f"File downloaded successfully to {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while downloading the file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def collect_download_info(num_models, num_lora_files):
    # Collect download information for SDXL models
    model_download_info = []
    for i in range(num_models):
        url = input(f"Paste SDXL Checkpoint URL #{i + 1} Here: ")
        user_filename = input(f"Enter the filename for Checkpoint #{i + 1} (including .ckpt or .safetensors extension): ")
        model_download_info.append((url, user_filename))

    # Collect download information for Lora files
    lora_download_info = []
    for i in range(num_lora_files):
        lora_url = input(f"Paste Lora File URL #{i + 1} Here: ")
        lora_name = input(f"Enter the filename for Lora File #{i + 1} (including .safetensors extension): ")
        lora_download_info.append((lora_url, lora_name))

    return model_download_info, lora_download_info

# Mount GDrive
mount_gdrive()

# Use pip to install pygit
!pip install pygit2==1.12.2

if not os.path.exists('/content/drive/MyDrive/Github'):
  os.makedirs('/content/drive/MyDrive/Github')

# Change Directory
%cd /content/drive/MyDrive/Github

# Clone the Fooocus repo
!git clone https://github.com/lllyasviel/Fooocus.git

# Change Directory
%cd /content/drive/MyDrive/Github/Fooocus

# Make sure we are running larest version if already installed
!git pull

# Ask the user how many models and Lora files they want to download
num_models = int(input("How Many SDXL Checkpoints Would You Like To Download (Maximum 4): "))
if num_models > 4:
    print("Maximum allowed is 4.")
    num_models = 4

num_lora_files = int(input("How Many Lora Files Would You Like To Download: "))

# Collect download information for SDXL models and Lora files
model_download_info, lora_download_info = collect_download_info(num_models, num_lora_files)

# Specify the folder path where you want to save the checkpoint files
checkpoint_folder_path = "/content/drive/MyDrive/Github/Fooocus/models/checkpoints"

# Download all requested SDXL checkpoint files
for url, user_filename in model_download_info:
    download_file_from_url(url, checkpoint_folder_path, user_filename)

# Specify the folder path where you want to save the Lora files
lora_folder_path = "/content/drive/MyDrive/Github/Fooocus/models/loras"

# Download all requested Lora files
for lora_url, lora_name in lora_download_info:
    download_file_from_url(lora_url, lora_folder_path, lora_name)

# Define the new output folder path
new_output_folder = '/content/drive/MyDrive/Images/Fooocus'

# Check if the new output folder exists, and create it if it doesn't
if not os.path.exists(new_output_folder):
    os.makedirs(new_output_folder)
    print(f"Created the new output folder: {new_output_folder}")

# Define the symbolic link path
symbolic_link_path = '/content/drive/MyDrive/Github/Fooocus/outputs'

# Check if the symbolic link exists, and create it if it doesn't
if not os.path.exists(symbolic_link_path):
    os.symlink(new_output_folder, symbolic_link_path)
    print(f"Created the symbolic link to the new output folder.")

# Start Fooocus and check for updates
%cd /content/drive/MyDrive/Github/Fooocus/
!python entry_with_update.py --share


Mounting Google Drive...
Mounted at /content/drive
Google Drive successfully mounted.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.0 MB/s eta 0:00:00
/content/drive/MyDrive/Github
fatal: destination path 'Fooocus' already exists and is not an empty directory.
/content/drive/MyDrive/Github/Fooocus
Already up to date.
How Many SDXL Checkpoints Would You Like To Download (Maximum 4): 0
How Many Lora Files Would You Like To Download: 0
/content/drive/MyDrive/Github/Fooocus
Already up-to-date
Update succeeded.
[System ARGV] ['entry_with_update.py', '--share']
Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Fooocus version: 2.2.0
Error checking version for torchsde: No package metadata was found for torchsde
Installing requirements
Traceback (most recent call last):
  File "/content/drive/MyDrive/Github/Fooocus/entry_with_update.py", line 46, in <module>
    from launch import *
  File "/content/drive/MyDrive/Github/Fooocus/launch.py", line 127, in <module>
    from 